In [ ]:
import dask
from dask import dataframe
import os
from aperturedb import EntityDataCSV
from dask.distributed import Client, LocalCluster, performance_report


batchsize = 4000
numthreads = 14

cluster = LocalCluster(n_workers=numthreads)
client = Client(cluster)
dask.config.set(scheduler="distributed")

FILENAME = os.path.join('see2.out')

ratings = dataframe.read_csv(
        FILENAME, blocksize=os.path.getsize(FILENAME) // numthreads)


In [ ]:
import dask
from dask import dataframe
import os
from aperturedb import EntityDataCSV
from dask.distributed import Client, SSHCluster, performance_report


batchsize = 4000
numthreads = 8

cluster = SSHCluster(
    ["localhost"],
    worker_options = {"n_workers": numthreads},
    connect_options = { "username": "ubuntu"}
)
client = Client(cluster)
dask.config.set(scheduler="distributed")

FILENAME = os.path.join('see2.out')

ratings = dataframe.read_csv(
        FILENAME, blocksize=os.path.getsize(FILENAME) // numthreads)


In [ ]:
import dask
from dask import dataframe
import os
from aperturedb import EntityDataCSV
from dask.distributed import Client, SSHCluster, performance_report


client = Client("localhost:8786")
dask.config.set(scheduler="distributed")



In [ ]:
from aperturedb import Connector

batchsize = 4000
numthreads = 16
FILENAME = os.path.join('see2.out')

ratings = dataframe.read_csv(
        FILENAME, blocksize=os.path.getsize(FILENAME) // numthreads)


# DB_HOST = "localhost"
DB_HOST = "www.walmart.on.aperturedata.io"

DB_PORT = 55555
DB_USER = "admin"
# DB_PASSWORD = "admin"
DB_PASSWORD = "t.QH[*XR0_^.o0.;wH"

def create_connector():
    return Connector.Connector(DB_HOST, DB_PORT, user=DB_USER, password=DB_PASSWORD)


def process(df):        
    # import dbinfo
    from aperturedb.ParallelLoader import ParallelLoader
    db = create_connector()
    loader = ParallelLoader(db)
    count = 0
    for i in range(0, len(df), batchsize):
        end = min(i + batchsize, len(df))
        batch = df[i:end]
        data = EntityDataCSV.EntityDataCSV(filename="", df=batch)
        loader.ingest(data, batchsize=len(batch), numthreads=1)
        count += 1
    print(f"len(df) = {len(df)}, count = {count}")

with performance_report(filename="ingestion_report.html"):
    ratings.map_partitions(process).compute()
